In [3]:
import json
import ollama

# Cargar los datos de música y personalidades
def load_music_data():
    with open('data/music_data.json', encoding='utf-8') as file:
        music_data = json.load(file)
    with open('data/genres_personalities.json', encoding='utf-8') as file:
        personalities_data = json.load(file)
    with open('data/mood_genres.json', encoding='utf-8') as file:
        mood_genres = json.load(file)
    return music_data, personalities_data, mood_genres

music_data, personalities_data, mood_genres = load_music_data()

# Obtener los géneros asociados a una personalidad
def get_genres_by_personality(personality, personalities_data):
    genres = []
    for genre, personalities in personalities_data.items():
        if personality.lower() in [p.lower() for p in personalities]:
            genres.append(genre)
    return genres

# Combinar géneros de personalidad y estado de ánimo
def get_combined_genres(personality_genres, mood_genres, mood):
    mood_related_genres = mood_genres.get(mood, [])
    combined_genres = list(set(personality_genres).intersection(set(mood_related_genres)))
    return combined_genres if combined_genres else list(set(personality_genres + mood_related_genres))

# Obtener recomendaciones basadas en los géneros combinados
def retrieve_data(data, genres):
    recommendations = {}
    artists_seen = set()
    for genre in genres:
        for artist in data:
            if artist['name'] in artists_seen:
                continue
            for album in artist['albums']:
                if genre.lower() in [g.lower() for g in album['genres']]:
                    if artist['name'] not in artists_seen:
                        recommendations[artist['name']] = {
                            'song': album['name'],
                            'artist': artist['name'],
                            'bio': artist['bio']
                        }
                        artists_seen.add(artist['name'])
                        break
            if len(recommendations) >= 10:
                break
        if len(recommendations) >= 10:
            break
    return recommendations

# Generar respuesta de recomendaciones
def generate_response(user_personality, user_mood):
    personality_genres = get_genres_by_personality(user_personality, personalities_data)
    combined_genres = get_combined_genres(personality_genres, mood_genres, user_mood)
    relevant_data = retrieve_data(music_data, combined_genres)
    response = "Aquí están tus recomendaciones de canciones y artistas basadas en tu personalidad y estado de ánimo:\n"
    for i, (artist, info) in enumerate(relevant_data.items(), start=1):
        response += f"{i}. Canción: {info['song']}, Artista: {artist}\n"
    return response, relevant_data

def query_artist_bio(recommendations):
    while True:
        artist_query = input("\n¿De qué artista deseas saber más? Ingresa el número correspondiente al artista: ")
        try:
            artist_index = int(artist_query) - 1
            if 0 <= artist_index < len(recommendations):
                artist_name = list(recommendations.keys())[artist_index]
                artist_bio = recommendations[artist_name]['bio']
                print(f"\nBiografía de {artist_name}:\n{artist_bio}")
                user_question = input(f"\n¿Qué más te gustaría saber sobre {artist_name}? Escribe tu pregunta: ")
                prompt = f"Usuario pregunta sobre {artist_name}: {user_question}\nBiografía: {artist_bio}\nRespuesta:"
                response = ollama.chat(model='phi3', messages=[{'role': 'user', 'content': prompt}])
                print(f"\nRespuesta: {response['message']['content']}")
                break
            else:
                print("Número de artista inválido. Por favor, ingresa un número válido.")
        except ValueError:
            print("Entrada inválida. Por favor, ingresa un número válido.")

# Ejemplo de uso
user_personality = input("Ingresa tu personalidad (extrovertido, introvertido, creativo, aventurero): ")
user_mood = input("¿Cuál es tu ánimo musical ahora? (happy, sad, energetic, relaxed, angry, romantic): ")
response, recommendations = generate_response(user_personality, user_mood)
print(response)
query_artist_bio(recommendations)  # Función para consultar sobre la biografía del artista con ollama


Aquí están tus recomendaciones de canciones y artistas basadas en tu personalidad y estado de ánimo:
1. Canción: Of Beyond, Artista: Adventure Time
2. Canción: Satie: Complete Piano Works, Vol. 2/2, Artista: Erik Satie
3. Canción: III, Artista: BADBADNOTGOOD
4. Canción: We Believe, Artista: Good Charlotte
5. Canción: Details (UK Version), Artista: Frou Frou
6. Canción: Trainspotting, Artista: Lou Reed

Entrada inválida. Por favor, ingresa un número válido.

Biografía de Erik Satie:

Alfred Éric Leslie Satie (Honfleur, 17 May 1866–Paris, 1 July 1925) was a French composer, pianist, and writer.

Over the years, Erik Satie was described as a precursor of movements and styles as varied as Impressionism, Neo-Classicism, Dada, Surrealism, Conceptual art, minimalism, and ambient music. As well as taking the first steps towards techniques such as prepared piano and music-to-film synchronisation, Satie became one of the first musicians to perform a cameo appearance in a film, in 1924's Entr'act